# Gen dataSet  
conclude:  
1. Promoter/Enhancer 's Sequence index, bases Sequence,kmer of Sequence  
2. label and cell types

# Result  
you can't not combine different cell type datset to one !  
It's the wrong way!

In [2]:
import os
import csv
import numpy as np
from pyfasta import Fasta
genome = Fasta('/home/yinqijin/WorkSpace/DataHub/genome.fa')

In [27]:
#获得ｋｍｅｒ的函数
d2 = {'a':0, 'A':0, 'g':1, 'G':1, 'c':2, 'C':2, 't':3, 'T':3, 'N':0, 'n':0}
def seq_to_kspec(seq):
    mat = np.zeros((4096,1))
    k = 0
    if len(seq)<6:
        return mat
    for i in range(6):
        k = k*4 + d2[seq[i]]
    mat[k]+=1  
    for i in range(6,len(seq)):
        k = k - 4**5*d2[seq[i-6]]
        k = k*4 + d2[seq[i]]
        mat[k] += 1  
    return mat

In [59]:
baseroot = '/home/zengwanwen/enhancer/targetfinder/paper/targetfinder'
excel_name =[]

#method2. 读取部分并计算新键
#-------------------------------
PESeq={}
PESeq['Pro-index']=[]
PESeq['Pro-Seq']=[]
PESeq['Pro-Kmer']=[]
PESeq['Ehr-index']=[]
PESeq['Ehr-Seq']=[]
PESeq['Ehr-Kmer']=[]
PESeq['label'] =[]
PESeq['Cell-Type']=[]
#--------------------------------
    
    
#读取每个pairs.csv的表头
for sub_dir in os.listdir(baseroot):
    cell_type_dir = os.path.join(baseroot,sub_dir)
    print cell_type_dir
    with open(cell_type_dir+'/output-epw/pairs.csv') as csvfile:
        for line in  csv.reader(csvfile):
            #print line
            excel_name=line
            break
    
    #新建初始化变量
    #method1.读取全部内容
    #----------------------------
    #PESeq = {}
    #for name in excel_name:
    #    PESeq[name] = []
    #---------------------------
    
    Pos_num =0
    Neg_num =0
    
    #读取CSV内容并结构化
    with open(cell_type_dir+'/output-epw/pairs.csv') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Add values of every keys to the csvdata for every row
            if abs(int(row['enhancer_start'] )-int( row['enhancer_end'])) <200 :
                if     row['label']==0:
                    print row
                    continue
            
            #正负样本均衡
            if  row['label']==str(1):
                Pos_num+=1
            elif  row['label']==str(0) :
                Neg_num+=1  
            if Neg_num>Pos_num:
                break
            #method1.读取全部内容   
            #---------------------------
            #for item in csvkeys:
            #    csvdata[item].append(row[item])
            #------------------------------
            #method2. 读取部分并计算新键
            #-------------------------------
            pro_index = [row['promoter_chrom'],row['promoter_start'],row['promoter_end']]   
            ehr_index = [row['enhancer_chrom'],row['enhancer_start'] ,row['enhancer_end']]
            pro_seq =  genome[row['promoter_chrom']][int(row['promoter_start']) :int(row['promoter_end'])] .upper()   
            pro_seq = str(pro_seq)
            ehr_seq =  genome[row['enhancer_chrom'] ][int(row['enhancer_start']) : int(row['enhancer_end'])].upper()    
            ehr_seq = str(ehr_seq)

            PESeq['Pro-index'].append(pro_index)
            PESeq['Pro-Seq'].append(pro_seq)
            PESeq['Pro-Kmer'].append(seq_to_kspec(pro_seq))
            PESeq['Ehr-index'].append(ehr_index)
            PESeq['Ehr-Seq'].append(ehr_seq)
            PESeq['Ehr-Kmer'].append(seq_to_kspec(ehr_seq))
            PESeq['label'].append(row['label'])
            PESeq['Cell-Type'].append(sub_dir)
            #--------------------------------

print 'Finished!'
    
                

/home/zengwanwen/enhancer/targetfinder/paper/targetfinder/GM12878
/home/zengwanwen/enhancer/targetfinder/paper/targetfinder/HeLa-S3
/home/zengwanwen/enhancer/targetfinder/paper/targetfinder/NHEK
/home/zengwanwen/enhancer/targetfinder/paper/targetfinder/IMR90
/home/zengwanwen/enhancer/targetfinder/paper/targetfinder/K562
/home/zengwanwen/enhancer/targetfinder/paper/targetfinder/combined
/home/zengwanwen/enhancer/targetfinder/paper/targetfinder/HUVEC


In [54]:
#Test their type
for item in PESeq.keys():
    print item,type(PESeq[item]),type(PESeq[item][0])

Ehr-Kmer <type 'list'> <type 'numpy.ndarray'>
Ehr-Seq <type 'list'> <type 'unicode'>
Pro-index <type 'list'> <type 'list'>
Pro-Kmer <type 'list'> <type 'numpy.ndarray'>
Cell-Type <type 'list'> <type 'str'>
Pro-Seq <type 'list'> <type 'unicode'>
Ehr-index <type 'list'> <type 'list'>
label <type 'list'> <type 'str'>


In [36]:
import hickle

In [60]:
hickle.dump(PESeq,'./Gen_data/11_Gen_Pro_Ehr.hkl')

In [64]:
Pos_num=0
Neg_num=0
for item in PESeq['label']:
    if item == str(0):
        Neg_num +=1
    if item == str(1):
        Pos_num +=1
        
assert Pos_num == Neg_num
print Pos_num,Neg_num


11876 11876
